In [1]:
from pkg.utils.utils_python3 import *
import random

CONVERTED_PATH = "./data/converted"
SCENE_FILENAME = "scene.pkl"
JOINT_NUM = 13
gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(os.listdir(CONVERTED_PATH))
trainset_num = None
if trainset_num is not None:
    TRAINSET_LIST =  DATASET_LIST[:trainset_num]
    TESTSET_LIST = DATASET_LIST[trainset_num:]
else:
    TRAINSET_LIST = ['20201212-232318', '20201213-061207', '20201214-165211']
    TESTSET_LIST = ['20201208-121454']
    
# trainset
SCENE_TUPLE_LIST = []
for DATASET in TRAINSET_LIST:
    CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
    #Iterate world
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    gtimer.reset()
    for WORLD in WORLD_LIST:
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        # Iterate scene
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
                for i_act in range(N_action):
                    SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                    #gtimer.tic("load_scene_data")
                    #scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                    #gtimer.toc("load_scene_data")
train_set = SCENE_TUPLE_LIST
N_train = len(train_set)

# trainset
SCENE_TUPLE_LIST = []
for DATASET in TESTSET_LIST:
    CURRENT_PATH = os.path.join(CONVERTED_PATH, DATASET)
    #Iterate world
    WORLD_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(CURRENT_PATH)))
    gtimer.reset()
    for WORLD in WORLD_LIST:
        WORLD_PATH = os.path.join(CURRENT_PATH, WORLD)
        # Iterate scene
        SCENE_LIST = sorted(filter(lambda x: not x.endswith(".json"), os.listdir(WORLD_PATH)))
        for SCENE in SCENE_LIST:
            SCENE_PATH = os.path.join(WORLD_PATH, SCENE)
            ACTION_LIST = sorted(filter(lambda x: x != SCENE_FILENAME, os.listdir(SCENE_PATH)))
            for ACTION in ACTION_LIST:
                N_action = get_action_count(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION)
                for i_act in range(N_action):
                    SCENE_TUPLE_LIST.append((CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM))
                    #gtimer.tic("load_scene_data")
                    #scene_data, success, skey = load_scene_data(CONVERTED_PATH, DATASET, WORLD, SCENE, ACTION, i_act, JOINT_NUM)
                    #gtimer.toc("load_scene_data")
test_set = SCENE_TUPLE_LIST
N_test = len(test_set)
print(gtimer)

In [2]:
joint_num = 13
N_vtx_box = 3*8
N_mask_box = 1
N_joint_box = joint_num
N_label_box = N_vtx_box+N_mask_box+N_joint_box
N_vtx_cyl = 3*2+1
N_mask_cyl = 1
N_joint_cyl = joint_num
N_label_cyl = N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_vtx_init = 3*8
N_mask_init = 1
N_joint_init = joint_num
N_label_init = N_vtx_init+N_mask_init+N_joint_init
N_vtx_goal = 3*8
N_mask_goal = 1
N_joint_goal = joint_num
N_label_goal = N_vtx_goal+N_mask_goal+N_joint_goal
N_joint_label = 6*joint_num
N_cell_label = N_label_box+N_label_cyl+N_label_init+N_label_goal + N_joint_label
N_BEGIN_CYL = N_vtx_box+N_mask_box+N_joint_box
N_BEGIN_INIT = N_BEGIN_CYL+N_vtx_cyl+N_mask_cyl+N_joint_cyl
N_BEGIN_GOAL = N_BEGIN_INIT+N_vtx_init+N_mask_init+N_joint_init
def separate_dat(scene_data):
    cbox = scene_data[:,:,:,:, :N_vtx_box]
    cbox_m = scene_data[:,:,:,:, N_vtx_box]
    cbox_j = scene_data[:,:,:,:, N_vtx_box+1:N_vtx_box+1+N_joint_box]
    ccyl = scene_data[:,:,:,:, N_BEGIN_CYL:N_BEGIN_CYL+N_vtx_cyl]
    ccyl_m = scene_data[:,:,:,:, N_BEGIN_CYL+N_vtx_cyl]
    ccyl_j = scene_data[:,:,:,:, N_BEGIN_CYL+N_vtx_cyl+1:N_BEGIN_CYL+N_vtx_cyl+1+N_joint_cyl]
    ibox = scene_data[:,:,:,:, N_BEGIN_INIT:N_BEGIN_INIT+N_vtx_box]
    ibox_m = scene_data[:,:,:,:, N_BEGIN_INIT+N_vtx_box]
    ibox_j = scene_data[:,:,:,:, N_BEGIN_INIT+N_vtx_box+1:N_BEGIN_INIT+N_vtx_box+1+N_joint_init]
    gbox = scene_data[:,:,:,:, N_BEGIN_GOAL:N_BEGIN_GOAL+N_vtx_box]
    gbox_m = scene_data[:,:,:,:, N_BEGIN_GOAL+N_vtx_box]
    gbox_j = scene_data[:,:,:,:, N_BEGIN_GOAL+N_vtx_box+1:N_BEGIN_GOAL+N_vtx_box+1+N_joint_goal]
    joints = scene_data[:,:,:,:,-N_joint_label:]
    joints = np.reshape(joints, scene_data.shape[:4]+(-1,6))
    cbox = np.concatenate([cbox, 
                           (joints*np.expand_dims(cbox_j, axis=-1)).reshape(scene_data.shape[:4]+(-1,))], 
                          axis=-1)
    ccyl = np.concatenate([ccyl, 
                           (joints*np.expand_dims(ccyl_j, axis=-1)).reshape(scene_data.shape[:4]+(-1,))], 
                          axis=-1)
    ibox = np.concatenate([ibox, 
                           (joints*np.expand_dims(ibox_j, axis=-1)).reshape(scene_data.shape[:4]+(-1,))], 
                          axis=-1)
    gbox = np.concatenate([gbox, 
                           (joints*np.expand_dims(gbox_j, axis=-1)).reshape(scene_data.shape[:4]+(-1,))], 
                          axis=-1)
    return (cbox, np.expand_dims(cbox_m, axis=-1), 
            ccyl, np.expand_dims(ccyl_m, axis=-1), 
            ibox, np.expand_dims(ibox_m, axis=-1), 
            gbox, np.expand_dims(gbox_m, axis=-1))

In [3]:
import tensorflow as tf

from tensorflow.keras import layers as KL
from tensorflow.keras import Model
import datetime

class DenseBN(KL.Layer):
    def __init__(self, units, name, use_bias=True, activation="relu"):
        super(DenseBN, self).__init__()
        self.dense = KL.Dense(units, name='dense_' + name, use_bias=use_bias)
        self.bn = KL.BatchNormalization(name='bn_' + name)
        self.ac = KL.Activation(activation)
        
    def call(self, x, training=False):
        x = self.dense(x)
        x = self.bn(x, training=training)
        x = self.ac(x)
        return x
        

class IdentityBlock(KL.Layer):
    def __init__(self, kernel_size, filters, stage, block,
                 use_bias=True, ConvLayer=KL.Conv2D):
        super(IdentityBlock, self).__init__()
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, name=conv_name_base + '2a',
                               use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        x = self.add([x, input_tensor])
        x = self.ac3(x)
        return x
    
class IdentityBlock3D(IdentityBlock):
    def __init__(self, *args, **kwargs):
        super(IdentityBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)
    
    

class ConvBlock(KL.Layer):
    def __init__(self, kernel_size, filters, stage, block, strides=2,
                 use_bias=True, ConvLayer=KL.Conv2D):
        super(ConvBlock, self).__init__()
        nb_filter1, nb_filter2, nb_filter3 = filters
        conv_name_base = 'res' + str(stage) + block + '_branch'
        bn_name_base = 'bn' + str(stage) + block + '_branch'
        
        self.conv1 = ConvLayer(nb_filter1, 1, strides=strides, 
                               name=conv_name_base + '2a', use_bias=use_bias)
        self.bn1 = KL.BatchNormalization(name=bn_name_base + '2a')
        self.ac1 = KL.Activation('relu')
        
        self.conv2 = ConvLayer(nb_filter2, kernel_size, padding="same",
                               name=conv_name_base + '2b', use_bias=use_bias)
        self.bn2 = KL.BatchNormalization(name=bn_name_base + '2b')
        self.ac2 = KL.Activation('relu')
        
        self.conv3 = ConvLayer(nb_filter3, 1, name=conv_name_base + '2c',
                               use_bias=use_bias)
        self.bn3 = KL.BatchNormalization(name=bn_name_base + '2c')

        self.convs = ConvLayer(nb_filter3, 1, strides=strides,
                               name=conv_name_base + '1', use_bias=use_bias)
        self.bns = KL.BatchNormalization(name=bn_name_base + '1')
    
        self.add = KL.Add()
        self.ac3 = KL.Activation('relu', name='res' + str(stage) + block + '_out')
        
    def call(self, input_tensor, training=False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training=training)
        x = self.ac1(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.ac2(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)
        
        shortcut = self.convs(input_tensor)
        shortcut = self.bns(shortcut, training=training)

        x = self.add([x, shortcut])
        x = self.ac3(x)
        return x
    
class ConvBlock3D(ConvBlock):
    def __init__(self, *args, **kwargs):
        super(ConvBlock3D, self).__init__(*args, ConvLayer=KL.Conv3D, **kwargs)
    

class ResNet(KL.Layer):
    def __init__(self, architecture="resnet50", stage0=None, stage1=None, stage2=[64, 64, 256], stage3=[128, 128, 512], 
                 stage4=[256, 256, 1024], stage5=None, 
                 ConvLayer=KL.Conv2D, ZeroPadding=KL.ZeroPadding2D, MaxPool=KL.MaxPooling2D, 
                 ConvBlock=ConvBlock, IdentityBlock=IdentityBlock, input_size=(15,15,15), joint_num=13, batch_size=16):
        super(ResNet, self).__init__()
        """Build a ResNet graph.
            architecture: Can be resnet50 or resnet101
            stage5: recommanded default values = [512, 512, 2048]. If None, stage5 of the network is not created
        """
        assert architecture in ["resnet50", "resnet101"]
        self.stage1 = stage1 is not None
        self.stage5 = stage5 is not None
        self.stage0 = stage0 is not None
        self.input_size = input_size
        self.joint_num = joint_num
        self.batch_size = batch_size
        if self.stage0:
            self.pre_boxa = ConvBlock(3, stage0, stage=0, block='ba', strides=1)
            self.pre_boxb = IdentityBlock(3, stage0, stage=0, block='bb')
            self.pre_boxc = IdentityBlock(3, stage0, stage=0, block='bc')
            self.pre_cyla = ConvBlock(3, stage0, stage=0, block='ca', strides=1)
            self.pre_cylb = IdentityBlock(3, stage0, stage=0, block='cb')
            self.pre_cylc = IdentityBlock(3, stage0, stage=0, block='cc')
            self.pre_inia = ConvBlock(3, stage0, stage=0, block='ia', strides=1)
            self.pre_inib = IdentityBlock(3, stage0, stage=0, block='ib')
            self.pre_inic = IdentityBlock(3, stage0, stage=0, block='ic')
            self.pre_gola = ConvBlock(3, stage0, stage=0, block='ga', strides=1)
            self.pre_golb = IdentityBlock(3, stage0, stage=0, block='gb')
            self.pre_golc = IdentityBlock(3, stage0, stage=0, block='gc')
            self.concat = KL.Concatenate()
        
        if self.stage1:
            # Stage 1  output size = 1/1
            self.cb1a = ConvBlock(3, stage1, stage=1, block='a', strides=1)
            self.ib1b = IdentityBlock(3, stage1, stage=1, block='b')
            self.ib1c = IdentityBlock(3, stage1, stage=1, block='c')
            stride_2 = 2
        else:
            # Stage 1 original output size = 1/4
            self.cv1 = ConvLayer(64, 7, strides=2, 
                                 name='conv1', padding="same", use_bias=True)
            self.bn1 = KL.BatchNormalization(name='bn_conv1')
            self.ac1 = KL.Activation('relu')
            self.mp1 = MaxPool(3, strides=2, padding="same")
            stride_2 = 1
        
        # Stage 2  output size = 1/stride_2
        self.cb2a = ConvBlock(3, stage2, stage=2, block='a', strides=stride_2)
        self.ib2b = IdentityBlock(3, stage2, stage=2, block='b')
        self.ib2c = IdentityBlock(3, stage2, stage=2, block='c')
        
        # Stage 3  output size = 1/2
        self.cb3a = ConvBlock(3, stage3, stage=3, block='a')
        self.ib3b = IdentityBlock(3, stage3, stage=3, block='b')
        self.ib3c = IdentityBlock(3, stage3, stage=3, block='c')
        self.ib3d = IdentityBlock(3, stage3, stage=3, block='d')
        
        # Stage 4  output size = 1/2
        self.cb4a = ConvBlock(3, stage4, stage=4, block='a')
        block_count = {"resnet50": 5, "resnet101": 22}[architecture]
        self.ib4_list = []
        for i in range(block_count):
            self.ib4_list.append(
                IdentityBlock(3, stage4, stage=4, block=chr(98 + i))
            )
            
        # Stage 5  output size = 1/2
        if self.stage5:
            self.cb5a = ConvBlock(3, stage5, stage=5, block='a')
            self.ib5b = IdentityBlock(3, stage5, stage=5, block='b')
            self.ib5c = IdentityBlock(3, stage5, stage=5, block='c')

    def call(self, input_image, training=False):
        if self.stage0:
            cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = input_image
            cbox = self.pre_boxa(cbox, training=training)
            cbox = self.pre_boxb(cbox, training=training)
            cbox = self.pre_boxc(cbox, training=training)*cbox_m
            ccyl = self.pre_cyla(ccyl, training=training)
            ccyl = self.pre_cylb(ccyl, training=training)
            ccyl = self.pre_cylc(ccyl, training=training)*ccyl_m
            ibox = self.pre_inia(ibox, training=training)
            ibox = self.pre_inib(ibox, training=training)
            ibox = self.pre_inic(ibox, training=training)*ibox_m
            gbox = self.pre_gola(gbox, training=training)
            gbox = self.pre_golb(gbox, training=training)
            gbox = self.pre_golc(gbox, training=training)*gbox_m
            x = self.concat([cbox, ccyl, ibox, gbox])
        else:
            x = input_image
        # Stage 1
        if self.stage1:
            x = self.cb1a(x, training=training)
            x = self.ib1b(x, training=training)
            C1 = x = self.ib1c(x, training=training)
        else:
            x = self.cv1(x)
            x = self.bn1(x, training=training)
            x = self.ac1(x)
            C1 = x = self.mp1(x)
        
        # Stage 2
        x = self.cb2a(x, training=training)
        x = self.ib2b(x, training=training)
        C2 = x = self.ib2c(x, training=training)
        
        # Stage 3
        x = self.cb3a(x, training=training)
        x = self.ib3b(x, training=training)
        x = self.ib3c(x, training=training)
        C3 = x = self.ib3d(x, training=training)
        
        # Stage 4
        x = self.cb4a(x, training=training)
        for ib4 in self.ib4_list:
            x = ib4(x, training=training)
        C4 = x
            
        # Stage 5
        if self.stage5:
            x = self.cb5a(x, training=training)
            x = self.ib5b(x, training=training)
            C5 = x = self.ib5c(x, training=training)
            return [C1, C2, C3, C4, C5]
        else:
            return [C1, C2, C3, C4]
            
    
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [4]:
class ResNetModelTP(Model):
    def __init__(self, 
                 ConvLayer=KL.Conv3D, ZeroPadding=KL.ZeroPadding3D, MaxPool=KL.GlobalMaxPool3D, 
                 ConvBlock=ConvBlock3D, IdentityBlock=IdentityBlock3D):
        super(ResNetModelTP, self).__init__()
        self.resnet = ResNet(architecture="resnet50", 
                             stage0 = [64,64,128], stage1=[256, 256, 512], 
                             stage2=[256, 256, 512], stage3=[256, 256, 512], 
                             stage4=[256, 256, 512], stage5=[256, 256, 512], 
                             ConvLayer=ConvLayer, ZeroPadding=ZeroPadding, MaxPool=MaxPool, 
                             ConvBlock=ConvBlock, IdentityBlock=IdentityBlock)
        self.gp1 = MaxPool()
        self.gp2 = MaxPool()
        self.gp3 = MaxPool()
        self.gp4 = MaxPool()
        self.flatten = KL.Flatten()
        self.dens11 = DenseBN(512, "dens11")
        self.dens21 = DenseBN(512, "dens21")
        self.dens31 = DenseBN(512, "dens31")
        self.dens41 = DenseBN(512, "dens41")
        self.dens51 = DenseBN(512, "dens51")
        self.dens12 = KL.Dense(256) # DenseBN(512, "dens1")
        self.dens22 = KL.Dense(256) # DenseBN(512, "dens2")
        self.dens32 = KL.Dense(256) # DenseBN(512, "dens3")
        self.dens42 = KL.Dense(256) # DenseBN(512, "dens4")
        self.dens52 = KL.Dense(256) # DenseBN(512, "dens5")
        self.add = KL.Add()
        self.dens_int1 = DenseBN(128, "dens_int1")
        self.dens_int2 = DenseBN(64, "dens_int2")
        self.dens_out = KL.Dense(2)

    def call(self, x, training=False):
        C1, C2, C3, C4, C5 = self.resnet(x, training=training)
        f1 = self.dens11(self.gp1(C1), training=training)
        f1 = self.dens12(f1, training=training)
        f2 = self.dens21(self.gp2(C2), training=training)
        f2 = self.dens22(f2, training=training)
        f3 = self.dens31(self.gp3(C3), training=training)
        f3 = self.dens32(f3, training=training)
        f4 = self.dens41(self.gp4(C4), training=training)
        f4 = self.dens42(f4, training=training)
        f5 = self.dens51(self.flatten(C5), training=training)
        f5 = self.dens52(f5, training=training)
        x = self.add([f1,f2,f3,f4,f5])
        # x = f5
        x = self.dens_int1(x, training=training)
        x = self.dens_int2(x, training=training)
        x = self.dens_out(x)
        return x

# Create an instance of the model
model = ResNetModelTP()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)


In [6]:
EPOCHS_S = 0
EPOCHS_E = 80
BATCH_SIZE = 16
LOG_STEP = 100

for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(train_set)
    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in train_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = separate_dat(np.array(scene_batch, dtype=np.float32))
            train_step([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m], np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')

    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in test_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = separate_dat(np.array(scene_batch, dtype=np.float32))
            test_step([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m], np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.6050356030464172, Accuracy: 69.98587799072266, Test Loss: 0.5871066451072693, Test Accuracy: 71.10969543457031
Epoch 2, Loss: 0.5882013440132141, Accuracy: 71.01309967041016, Test Loss: 0.6037904024124146, Test Accuracy: 70.67921447753906
Epoch 3, Loss: 0.5776786208152771, Accuracy: 71.19286346435547, Test Loss: 0.5792039632797241, Test Accuracy: 72.1141586303711
Epoch 4, Loss: 0.538532018661499, Accuracy: 73.40460205078125, Test Loss: 0.5715247392654419, Test Accuracy: 68.78189086914062
Epoch 5, Loss: 0.5208334922790527, Accuracy: 74.88443756103516, Test Loss: 0.5665736794471741, Test Accuracy: 72.14604949951172
Epoch 6, Loss: 0.510137677192688, Accuracy: 75.10592651367188, Test Loss: 0.5318417549133301, Test Accuracy: 75.03189086914062
Epoch 7, Loss: 0.498340368270874, Accuracy: 75.8089370727539, Test Loss: 0.6081823110580444, Test Accuracy: 66.80484771728516
Epoch 8, Loss: 0.4856240749359131, Accuracy: 76.60503387451172, Test Loss: 0.5485255122184753, Test Accuracy:

KeyboardInterrupt: 

### train log
* dense bn for end stage: saturated output
* dense batch norm for each stage: 74% for train, saturated for test
* dense linear for each stage: 73% for train, saturated for test
* dense bn relu - dense linear for each stage: 74% for train, saturated for test
* dense bn relu - dense linear for each stage (size down): 74% for train, saturated for test
* separate input - dense bn relu - dense linear for each stage : 50epoch - 98.5% for train, max 74% for test
* dense^2 relu/sigmoid for each stage: 
* dense^2 relu/sigmoid x2 for each stage: 

# test mnist

In [2]:
import numpy as np
class ResNetModelTest(Model):
    def __init__(self):
        super(ResNetModelTest, self).__init__()
        self.resnet = ResNet(architecture="resnet50", stage1=[256, 256, 512], 
                             stage2=[256, 256, 512], stage3=[512, 512, 1024], 
                             stage4=[512, 512, 1024], stage5=[512, 512, 1024])
        self.gp1 = KL.GlobalMaxPool2D()
        self.gp2 = KL.GlobalMaxPool2D()
        self.gp3 = KL.GlobalMaxPool2D()
        self.gp4 = KL.GlobalMaxPool2D()
        self.flatten = KL.Flatten()
        self.dens1 = DenseBN(512, "dens1")
        self.dens2 = DenseBN(512, "dens2")
        self.dens3 = DenseBN(512, "dens3")
        self.dens4 = DenseBN(512, "dens4")
        self.dens5 = DenseBN(512, "dens5")
        self.add = KL.Add()
        self.dens_int = DenseBN(128, "dens_int")
        self.dens_out = KL.Dense(2)

    def call(self, x, training=False):
        C1, C2, C3, C4, C5 = self.resnet(x, training=training)
        # f1 = train_lossself.dens1(self.gp1(C1), training=training)
        # f2 = self.dens2(self.gp2(C2), training=training)
        # f3 = self.dens3(self.gp3(C3), training=training)
        # f4 = self.dens4(self.gp4(C4), training=training)
        f5 = self.dens5(self.flatten(C5), training=training)
        # x = self.add([f1,f2,f3,f4,f5])
        x = f5
        x = self.dens_int(x, training=training)
        x = self.dens_out(x)
        return x

# Create an instance of the model
model = ResNetModelTest()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [6]:
mnist = tf.keras.datasets.mnist

dat = mnist.load_data()


In [7]:
x_train, y_train = dat[0]
x_test, y_test = dat[1]

In [8]:
x_train_res = np.expand_dims(np.repeat(np.expand_dims(np.pad(x_train[:,0::2, 0::2], [[0,0],[0,1],[0,1]]), axis=-1), 15, axis=-1), axis=-1)
x_test_res = np.expand_dims(np.repeat(np.expand_dims(np.pad(x_test[:,0::2, 0::2], [[0,0],[0,1],[0,1]]), axis=-1), 15, axis=-1), axis=-1)

In [9]:
# x_train_res = np.expand_dims(np.pad(x_train[:,0::2, 0::2], [[0,0],[0,1],[0,1]]), axis=-1)
# x_test_res = np.expand_dims(np.pad(x_test[:,0::2, 0::2], [[0,0],[0,1],[0,1]]), axis=-1)

In [10]:
EPOCHS = 5
BATCH_SIZE = 16
LOG_STEP = 100

N_train = len(x_train_res)
N_test = len(x_test_res)
for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    i_step = 0
    scene_batch, success_batch = [], []
    for _ in range(5000):
        scene_data, success, skey = x_train_res[i_step], y_train[i_step], None
        scene_batch.append(scene_data)
        success_batch.append(success>=5)
        i_step += 1
        if i_step%BATCH_SIZE==0:
            train_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')

    i_step = 0
    scene_batch, success_batch = [], []
    for _ in range(5000):
        scene_data, success, skey = x_test_res[i_step], y_test[i_step], None
        scene_batch.append(scene_data)
        success_batch.append(success>=5)
        i_step += 1
        if i_step%BATCH_SIZE==0:
            test_step(np.array(scene_batch, dtype=np.float32), np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

Epoch 1, Loss: 0.3269312381744385, Accuracy: 85.61698913574219, Test Loss: 0.33408501744270325, Test Accuracy: 85.59695434570312
Epoch 2, Loss: 0.1531522274017334, Accuracy: 94.39102172851562, Test Loss: 0.460206001996994, Test Accuracy: 83.95433044433594
Epoch 3, Loss: 0.0960620790719986, Accuracy: 96.61457824707031, Test Loss: 0.3936728239059448, Test Accuracy: 85.49679565429688


KeyboardInterrupt: 

In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [6]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.14230504631996155, Accuracy: 95.68333435058594, Test Loss: 0.05939216911792755, Test Accuracy: 97.91999816894531
Epoch 2, Loss: 0.04611682891845703, Accuracy: 98.52166748046875, Test Loss: 0.050787363201379776, Test Accuracy: 98.18999481201172
Epoch 3, Loss: 0.024921394884586334, Accuracy: 99.20832824707031, Test Loss: 0.0528109110891819, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 0.016222544014453888, Accuracy: 99.46666717529297, Test Loss: 0.05345192924141884, Test Accuracy: 98.5
Epoch 5, Loss: 0.010034429840743542, Accuracy: 99.66666412353516, Test Loss: 0.05353892967104912, Test Accuracy: 98.41999816894531


In [10]:
print(images.shape, labels.shape)

(32, 28, 28, 1) (32,)


In [11]:
labels

<tf.Tensor: shape=(32,), dtype=uint8, numpy=
array([3, 4, 6, 2, 3, 4, 8, 6, 4, 2, 6, 3, 9, 0, 1, 0, 8, 3, 6, 0, 8, 0,
       2, 2, 2, 0, 2, 2, 1, 3, 3, 7], dtype=uint8)>